In [1]:
# Import libraries
import os
import pandas as pd
import numpy as np
import geopandas as gpd
import h5py
import boto.s3
import glob
import boto3
from zipfile import ZipFile
import shutil
import io

In [2]:
import pyreadr
import openmatrix as omx

Merging wait times from skims

In [3]:
# This empty df will contain the median waits
df_median_pooled = pd.DataFrame()
df_median_solo = pd.DataFrame()

In [4]:
# Select time period
#period = 'pm'
period_list = ['am', 'pm', 'ev', 'md', 'ea']

# options: 'am', 'pm', 'ev', 'md', 'ea'

In [5]:
for period in period_list:
    combined_csv_pooled = pd.read_csv(f'all_waits_pooled_{period}.csv')
    combined_csv_solo = pd.read_csv(f'all_waits_solo_{period}.csv')
    s = pd.Series(range(3,45))
    s2 = (s.repeat(8)).to_list()
    combined_csv_pooled['run'] = s2
    combined_csv_solo['run'] = s2
    df_pooled = combined_csv_pooled.pivot(index='run', columns='Unnamed: 0', values='1')
    df_solo = combined_csv_solo.pivot(index='run', columns='Unnamed: 0', values='1')
    median_wait_pooled = df_pooled['50%']
    median_wait_solo = df_solo['50%']
    df_median_pooled.insert(loc=len(df_median_pooled.columns), column=f'{period}_median_pooled', value = median_wait_pooled)
    df_median_solo.insert(loc=len(df_median_solo.columns), column=f'{period}_median_solo', value = median_wait_solo)

In [6]:
df_median_all = pd.merge(df_median_pooled, df_median_solo, how="left", on=["run"])
df_median_all['index_col'] = range(1, len(df_median_all) + 1)
df_median_all['run_col'] = df_median_all.index

In [7]:
# This dataframe contains, by scenario run, the median waiting time for RH pooled and RH solo, by time period.
df_median_all

,am_median_pooled,pm_median_pooled,ev_median_pooled,md_median_pooled,ea_median_pooled,am_median_solo,pm_median_solo,ev_median_solo,md_median_solo,ea_median_solo,index_col,run_col
run,,,,,,,,,,,,
3,2.336867,2.355212,2.861287,2.607493,2.336867,3.361111,3.575000,3.183333,3.565115,3.361111,1,3
4,2.565692,2.600000,3.100000,2.795681,2.565692,3.174236,3.397014,3.133333,3.449851,3.174236,2,4
5,2.620296,2.742675,2.971164,2.888016,2.620296,2.857414,3.025000,2.927778,3.305787,2.857414,3,5
6,2.288102,2.256888,2.679104,2.556292,2.288102,2.717361,2.998958,2.589583,3.068519,2.717361,4,6
7,2.020361,2.055312,2.598273,2.390337,2.020361,2.753471,2.936932,2.635186,3.044445,2.753471,5,7
8,1.235631,1.312462,2.024999,1.575189,1.235631,2.326137,2.748016,2.453567,2.886279,2.326137,6,8
9,0.700000,0.717090,1.134607,0.741117,0.700000,2.001852,2.602778,2.122549,2.526221,2.001852,7,9
10,2.046088,2.095170,2.782404,2.382018,2.046088,3.133333,3.472909,3.014583,3.459722,3.133333,8,10
11,1.845555,1.825380,2.639584,2.204688,1.845555,3.040000,3.323959,3.043055,3.370000,3.040000,9,11


Opening summary tables

In [8]:
df_consol=pd.read_csv("summaries_condensed_28Feb.csv")
df_consol_t=df_consol.T
df=df_consol_t
new_header = df.iloc[0] #grab the first row for the header
df = df[1:] #take the data less the header row
df.columns = new_header #set the header row as the df header
df = df[:-1] #drop the last row
df['index_col'] = range(1, len(df) + 1)
df.head()

Variable,Number of fleets,Total fleet size,RH Pooled Price,RH Solo Price,Simulated Agents,Trips per Agent AV,Trip Vehicle Total,Trip Est Total,Trip Mode Total,Trip Replanning Total,...,Activities AV per person eatout,Activities AV per person escort,Activities AV per person othdiscr,Activities AV per person othmaint,Activities AV per person school,Activities AV per person shopping,Activities AV per person social,Activities AV per person univ,Activities AV per person work,index_col
Baseline,1,100%,100%,100%,1937432,3.837653657,17844548,7435193,7604885,169726,...,0.181533888,0.331747437,0.208376796,0.234005249,0.195663989,0.378355041,0.094250566,0.044597276,0.526242906,1
1Fl 1Pr 1Flz,2,100%,100%,100%,1937360,3.839640542,17848430,7438766,7606010,167283,...,0.181895416,0.331933714,0.208591063,0.23420693,0.195638866,0.37864916,0.094146272,0.044579629,0.526276535,2
5Fl 1Pr 1Flz,5,100%,100%,100%,1937424,3.839001685,17855388,7437774,7611958,174218,...,0.181651006,0.331946691,0.208309119,0.233997442,0.19568276,0.378870771,0.094238862,0.044607733,0.526228185,3
5Fl 1Pr 1.6Flz,5,164%,100%,100%,1937388,3.839239223,17846876,7438096,7608110,170051,...,0.181591917,0.331842667,0.208545733,0.23402798,0.195693893,0.37864021,0.094237189,0.044612127,0.526232226,4
5Fl 1Pr 2Flz,5,200%,100%,100%,1937426,3.839439545,17830954,7438630,7602653,164059,...,0.181856846,0.331766649,0.208518523,0.234009575,0.1956788,0.378741887,0.094186356,0.044602501,0.52617211,5


In [9]:
len(df)

42

In [10]:
len(df_median_all)

42

In [11]:
summaries_waits = pd.merge(df, df_median_all, on="index_col")

In [12]:
# This dataframe contains the wait times from the skims merged with the summary tables from Cristian
summaries_waits.head()

,Number of fleets,Total fleet size,RH Pooled Price,RH Solo Price,Simulated Agents,Trips per Agent AV,Trip Vehicle Total,Trip Est Total,Trip Mode Total,Trip Replanning Total,...,pm_median_pooled,ev_median_pooled,md_median_pooled,ea_median_pooled,am_median_solo,pm_median_solo,ev_median_solo,md_median_solo,ea_median_solo,run_col
0,1,100%,100%,100%,1937432,3.837653657,17844548,7435193,7604885,169726,...,2.355212,2.861287,2.607493,2.336867,3.361111,3.575000,3.183333,3.565115,3.361111,3
1,2,100%,100%,100%,1937360,3.839640542,17848430,7438766,7606010,167283,...,2.600000,3.100000,2.795681,2.565692,3.174236,3.397014,3.133333,3.449851,3.174236,4
2,5,100%,100%,100%,1937424,3.839001685,17855388,7437774,7611958,174218,...,2.742675,2.971164,2.888016,2.620296,2.857414,3.025000,2.927778,3.305787,2.857414,5
3,5,164%,100%,100%,1937388,3.839239223,17846876,7438096,7608110,170051,...,2.256888,2.679104,2.556292,2.288102,2.717361,2.998958,2.589583,3.068519,2.717361,6
4,5,200%,100%,100%,1937426,3.839439545,17830954,7438630,7602653,164059,...,2.055312,2.598273,2.390337,2.020361,2.753471,2.936932,2.635186,3.044445,2.753471,7


Opening Inexus summary tables

In [13]:
inexus=pd.read_csv('sf_2018_stacked_rh_fltsz_price_incRank_1_2_3.csv')

In [14]:
inexus

,incRank,n_agents,n_trips,Potential_INEXUS_mean,Potential_INEXUS_median,cost_BEAM_mean,cost_BEAM_median,cost_BEAM_sum,distance_bike_mean,distance_bike_median,...,waitTime_replanning_car_median,waitTime_replanning_car_hov2_median,waitTime_replanning_car_hov3_median,waitTime_replanning_drive_transit_median,waitTime_replanning_hov2_teleportation_median,waitTime_replanning_hov3_teleportation_median,waitTime_replanning_ride_hail_median,waitTime_replanning_ride_hail_pooled_median,waitTime_replanning_walk_median,waitTime_replanning_walk_transit_median
0,0,91811,319870,-0.620502,-0.367668,0.877347,0.341667,280636.854563,73.591278,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,118.0,0.0,0.0
1,1,87944,308814,-0.648142,-0.400426,0.905470,0.358061,279621.758581,72.442177,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,112.0,0.0,0.0
2,2,87362,321287,-0.665327,-0.424735,0.938598,0.376697,301559.310644,75.393038,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,107.0,0.0,0.0
3,3,96029,359786,-0.578462,-0.346745,0.999600,0.408441,359642.177341,65.588663,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,118.0,0.0,0.0
4,4,94883,355740,-0.581522,-0.349600,1.028149,0.427658,365753.736920,64.640784,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
877,16,92371,364566,-0.554504,-0.331923,1.741541,0.668759,634906.529641,52.212030,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
878,17,85124,334481,-0.556483,-0.333113,1.710566,0.670575,572151.726916,50.988010,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
879,18,94695,372634,-0.569288,-0.345549,1.721363,0.675854,641438.204127,50.281434,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
880,19,89513,350870,-0.547570,-0.326860,1.725924,0.681660,605575.084923,49.732606,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
# Every 21 rows corresponds to one scenario run
s3 = pd.Series(range(3,45))
s4 = (s.repeat(21)).to_list()

In [16]:
inexus['run_col'] = s4

In [17]:
full_merge = pd.merge(inexus, summaries_waits, on="run_col")

In [18]:
full_merge

,incRank,n_agents,n_trips,Potential_INEXUS_mean,Potential_INEXUS_median,cost_BEAM_mean,cost_BEAM_median,cost_BEAM_sum,distance_bike_mean,distance_bike_median,...,am_median_pooled,pm_median_pooled,ev_median_pooled,md_median_pooled,ea_median_pooled,am_median_solo,pm_median_solo,ev_median_solo,md_median_solo,ea_median_solo
0,0,91811,319870,-0.620502,-0.367668,0.877347,0.341667,280636.854563,73.591278,0.0,...,2.336867,2.355212,2.861287,2.607493,2.336867,3.361111,3.575000,3.183333,3.565115,3.361111
1,1,87944,308814,-0.648142,-0.400426,0.905470,0.358061,279621.758581,72.442177,0.0,...,2.336867,2.355212,2.861287,2.607493,2.336867,3.361111,3.575000,3.183333,3.565115,3.361111
2,2,87362,321287,-0.665327,-0.424735,0.938598,0.376697,301559.310644,75.393038,0.0,...,2.336867,2.355212,2.861287,2.607493,2.336867,3.361111,3.575000,3.183333,3.565115,3.361111
3,3,96029,359786,-0.578462,-0.346745,0.999600,0.408441,359642.177341,65.588663,0.0,...,2.336867,2.355212,2.861287,2.607493,2.336867,3.361111,3.575000,3.183333,3.565115,3.361111
4,4,94883,355740,-0.581522,-0.349600,1.028149,0.427658,365753.736920,64.640784,0.0,...,2.336867,2.355212,2.861287,2.607493,2.336867,3.361111,3.575000,3.183333,3.565115,3.361111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
877,16,92371,364566,-0.554504,-0.331923,1.741541,0.668759,634906.529641,52.212030,0.0,...,0.787544,0.842511,1.565978,0.866562,0.787544,3.034921,2.951281,3.213913,2.922860,3.034921
878,17,85124,334481,-0.556483,-0.333113,1.710566,0.670575,572151.726916,50.988010,0.0,...,0.787544,0.842511,1.565978,0.866562,0.787544,3.034921,2.951281,3.213913,2.922860,3.034921
879,18,94695,372634,-0.569288,-0.345549,1.721363,0.675854,641438.204127,50.281434,0.0,...,0.787544,0.842511,1.565978,0.866562,0.787544,3.034921,2.951281,3.213913,2.922860,3.034921
880,19,89513,350870,-0.547570,-0.326860,1.725924,0.681660,605575.084923,49.732606,0.0,...,0.787544,0.842511,1.565978,0.866562,0.787544,3.034921,2.951281,3.213913,2.922860,3.034921


In [19]:
full_merge.to_csv('inexus_waits_summary_tables.csv')